
このノートブックを実行するには、次の追加ライブラリが必要です。 Colab での実行は実験的なものであることに注意してください。問題がある場合は、Github の問題を報告してください。


In [ ]:
!pip install d2l==1.0.0-beta0



# 合成回帰データ

:label: `sec_synthetic-regression-data`

機械学習とは、データから情報を抽出することです。それでは、合成データから何が学べるのかと疑問に思うかもしれません。私たち自身が人工データ生成モデルに焼き付けたパターンについては本質的には気にしていないかもしれませんが、それでもそのようなデータセットは教育目的に役立ち、学習アルゴリズムの特性を評価し、実装が期待どおりに機能することを確認するのに役立ちます。たとえば、正しいパラメーターが*事前に*わかっているデータを作成した場合、モデルが実際にそれらを回復できることを検証できます。


In [1]:
%matplotlib inline
import random
import torch
from d2l import torch as d2l


## データセットの生成

この例では、簡潔にするために低次元で作業します。次のコード スニペットは、標準正規分布から抽出された 2 次元特徴を含む 1000 個の例を生成します。結果の計画行列 $\mathbf{X}$ は $\mathbb{R}^{1000 \times 2}$ に属します。*グラウンド トゥルース*線形関数を適用して各ラベルを生成し、加法ノイズ $\epsilon$ によってラベルを破損し、各例で独立して同一に描画します。

 ( **$$\mathbf{y}= \mathbf{X} \mathbf{w} + b + \mathbf\epsilon.$$** )

便宜上、$\epsilon$ は平均 $\mu= 0$、標準偏差 $\sigma = 0.01$ の正規分布から抽出されると仮定します。オブジェクト指向設計の場合、 `d2l.DataModule` ( :numref: `oo-design-data`で紹介) のサブクラスの`__init__`メソッドにコードを追加することに注意してください。追加のハイパーパラメータを設定できるようにすることをお勧めします。これは`save_hyperparameters()`で実現します。 `batch_size`後で決定されます。


In [2]:
class SyntheticRegressionData(d2l.DataModule):  #@save
    """Synthetic data for linear regression."""
    def __init__(self, w, b, noise=0.01, num_train=1000, num_val=1000,
                 batch_size=32):
        super().__init__()
        self.save_hyperparameters()
        n = num_train + num_val
        self.X = torch.randn(n, len(w))
        noise = torch.randn(n, 1) * noise
        self.y = torch.matmul(self.X, w.reshape((-1, 1))) + b + noise


以下では、真のパラメータを $\mathbf{w} = [2, -3.4]^\top$ および $b = 4.2$ に設定します。後で、推定されたパラメーターをこれらの*グラウンド トゥルース*値と照合して確認できます。


In [3]:
data = SyntheticRegressionData(w=torch.tensor([2, -3.4]), b=4.2)


[ **`features`の各行は $\mathbb{R}^2$ のベクトルで構成され、 `labels`の各行はスカラーです。** 】 最初のエントリーを見てみましょう。


In [4]:
print('features:', data.X[0],'\nlabel:', data.y[0])

features: tensor([-0.0499, -0.2817]) 
label: tensor([5.0533])



## データセットの読み取り

機械学習モデルのトレーニングには、多くの場合、データセット上で複数のパスを実行し、一度に 1 つのサンプルのミニバッチを取得する必要があります。このデータはモデルの更新に使用されます。これがどのように機能するかを説明するために、[ **`get_dataloader`メソッドを実装し]、** `add_to_class` (:numref: `oo-design-utilities`で導入) を介してそれを`SyntheticRegressionData`クラスに登録します。これは (**バッチ サイズ、特徴の行列、およびラベルのベクトルを受け取り、サイズ`batch_size`のミニバッチを生成します。** )そのため、各ミニバッチは特徴とラベルのタプルで構成されます。トレーニング モードと検証モードのどちらにいるかに注意する必要があることに注意してください。前者ではデータをランダムな順序で読み取る必要がありますが、後者の場合は、事前に定義された順序でデータを読み取ることができるため、デバッグの目的で重要です。


In [5]:
@d2l.add_to_class(SyntheticRegressionData)
def get_dataloader(self, train):
    if train:
        indices = list(range(0, self.num_train))
        # The examples are read in random order
        random.shuffle(indices)
    else:
        indices = list(range(self.num_train, self.num_train+self.num_val))
    for i in range(0, len(indices), self.batch_size):
        batch_indices = torch.tensor(indices[i: i+self.batch_size])
        yield self.X[batch_indices], self.y[batch_indices]


直感を構築するために、データの最初のミニバッチを調べてみましょう。特徴の各ミニバッチは、そのサイズと入力特徴の次元の両方を提供します。同様に、ラベルのミニバッチは、 `batch_size`で指定された一致する形状になります。


In [6]:
X, y = next(iter(data.train_dataloader()))
print('X shape:', X.shape, '\ny shape:', y.shape)

X shape: torch.Size([32, 2]) 
y shape: torch.Size([32, 1])



一見無害に見えますが、 `iter(data.train_dataloader())`の呼び出しは、Python のオブジェクト指向設計の威力を示しています。 `data`オブジェクトの作成*後に、* `SyntheticRegressionData`クラスにメソッドを追加したことに注意してください。それにもかかわらず、オブジェクトはクラスへの機能の*事後*追加から恩恵を受けます。

反復を通じて、データセット全体が使い果たされるまで、個別のミニバッチを取得します (これを試してください)。上記で実装された反復は教訓的な目的には適していますが、実際の問題で問題が発生する可能性がある点で非効率的です。たとえば、すべてのデータをメモリにロードし、大量のランダム メモリ アクセスを実行する必要があります。深層学習フレームワークに実装された組み込みイテレータはかなり効率的で、ファイルに保存されたデータ、ストリーム経由で受信したデータ、オンザフライで生成または処理されたデータなどのソースを処理できます。次に、組み込みイテレータを使用して同じメソッドを実装してみましょう。

## データローダーの簡潔な実装

独自のイテレータを作成する代わりに、[**フレームワーク内の既存の API を呼び出してデータをロードできます。** ] 前と同様に、特徴`X`とラベル`y`を持つデータセットが必要です。さらに、組み込みデータローダーで`batch_size`を設定し、サンプルのシャッフルを効率的に処理させます。


In [7]:
@d2l.add_to_class(d2l.DataModule)  #@save
def get_tensorloader(self, tensors, train, indices=slice(0, None)):
    tensors = tuple(a[indices] for a in tensors)
    dataset = torch.utils.data.TensorDataset(*tensors)
    return torch.utils.data.DataLoader(dataset, self.batch_size,
                                       shuffle=train)

In [8]:
@d2l.add_to_class(SyntheticRegressionData)  #@save
def get_dataloader(self, train):
    i = slice(0, self.num_train) if train else slice(self.num_train, None)
    return self.get_tensorloader((self.X, self.y), train, i)


新しいデータ ローダーは、より効率的でいくつかの機能が追加されている点を除いて、以前のデータ ローダーと同じように動作します。


In [9]:
X, y = next(iter(data.train_dataloader()))
print('X shape:', X.shape, '\ny shape:', y.shape)

X shape: torch.Size([32, 2]) 
y shape: torch.Size([32, 1])



たとえば、フレームワーク API によって提供されるデータ ローダーは組み込みの`__len__`メソッドをサポートしているため、その長さ、つまりバッチの数をクエリできます。


In [10]:
len(data.train_dataloader())

32


## まとめ

データ ローダーは、データのロードと操作のプロセスを抽象化する便利な方法です。このようにして、同じ機械学習*アルゴリズムで*、変更を加えることなく、さまざまな種類やソースのデータを処理できます。データ ローダーの優れた点の 1 つは、データ ローダーを構成できることです。たとえば、画像をロードし、それをトリミングしたり変更したりする後処理フィルターを使用する場合があります。したがって、データ ローダーを使用して、データ処理パイプライン全体を記述することができます。

モデル自体に関しては、2 次元線形モデルは、私たちが遭遇する可能性があるのと同じくらい単純なモデルです。これにより、データ量が不十分であることや方程式系が不完全に決定されていることを心配することなく、回帰モデルの精度をテストできます。これを次のセクションで有効に活用します。

## 演習
1. サンプルの数をバッチ サイズで割ることができない場合はどうなりますか。フレームワークの API を使用して別の引数を指定してこの動作を変更するにはどうすればよいですか?
1. パラメーター ベクトル`w`のサイズとサンプル数`num_examples`の両方が大きい、巨大なデータセットを生成したい場合はどうすればよいでしょうか?<ol><li>すべてのデータをメモリに保持できない場合はどうなるのでしょうか?
1. データがディスク上に保持されている場合、データをどのようにシャッフルしますか?あなたのタスクは、ランダムな読み取りまたは書き込みをあまり必要としない*効率的な*アルゴリズムを設計することです。ヒント:[擬似ランダム順列ジェネレーターを使用すると、](https://en.wikipedia.org/wiki/Pseudorandom_permutation)順列テーブルを明示的に保存しなくても再シャッフルを設計できます (引用: `Naor.Reingold.1999` )。



[ディスカッション](https://discuss.d2l.ai/t/6663)
